In [1]:
import tensorflow as tf

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 简单卷积层，为方便本章教程叙述，固定部分参数


def conv_layer(input,
               channels_in,    # 输入通道数
               channels_out,   # 输出通道数
               name='conv'):   # 名称
    with tf.name_scope(name):
        weights = tf.Variable(tf.truncated_normal([5, 5, channels_in, channels_out],
                                                  stddev=0.05), name='W')
        biases = tf.Variable(tf.constant(0.05, shape=[channels_out]), name='B')
        conv = tf.nn.conv2d(input, filter=weights, strides=[
                            1, 1, 1, 1], padding='SAME')
        act = tf.nn.relu(conv + biases)

        # 收集以下三个信息，统计直方图
        tf.summary.histogram('weights', weights)
        tf.summary.histogram('biases', biases)
        tf.summary.histogram('activations', act)
        return act


# 简化全连接层
def fc_layer(input, num_inputs, num_outputs, use_relu=True, name='fc'):
    with tf.name_scope(name):   # 在该名称作用域下的子变量
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs],
                                                  stddev=0.05), name='W')
        biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]), name='B')
        act = tf.matmul(input, weights) + biases

        if use_relu:
            act = tf.nn.relu(act)
        return act

# max pooling 层


def max_pool(input):
    return tf.nn.max_pool(input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST', one_hot=True)
x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
x_image = tf.reshape(x, [-1, 28, 28, 1])

conv1 = conv_layer(x_image, 1, 32, 'conv1')
pool1 = max_pool(conv1)

conv2 = conv_layer(pool1, 32, 64, 'conv2')
pool2 = max_pool(conv2)

flat_shape = pool2.get_shape()[1:4].num_elements()
flattened = tf.reshape(pool2, [-1, flat_shape])

fc1 = fc_layer(flattened, flat_shape, 1024, name='fc1')
logits = fc_layer(fc1, 1024, 10, use_relu=False, name='fc2')

# 计算交叉熵
with tf.name_scope("xent"):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))

# 使用Adam优化器来训练
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(
        learning_rate=1e-4).minimize(cross_entropy)

# 计算准确率
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(
        tf.argmax(y, axis=1), tf.argmax(logits, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
tf.summary.image('input', x_image, 3)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/MNIST\t10k-images-idx3-ubyte.gz
Extracting data/MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



<tf.Tensor 'input:0' shape=() dtype=string>

In [4]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [5]:
import os

In [6]:
tensorboard_dir = 'tensorboard/mnist3'   # 保存到新的目录
if not os.path.exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)

# 使用tf.summary.merge_all()，可以收集以上定义的所有信息
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(tensorboard_dir)
writer.add_graph(session.graph)

In [7]:
train_batch_size = 100

for i in range(2001):
    x_batch, y_batch = data.train.next_batch(train_batch_size)

    feed_dict = {x: x_batch, y: y_batch}

    if i % 5 == 0:   # 运行merger_summary，使用add_summary写入数据
        # 这里的feed_dict应该使用验证集，但是当前仅作为演示目的，在此不做修改
        s = session.run(merged_summary, feed_dict=feed_dict)
        writer.add_summary(s, i)

    if i % 500 == 0:
        train_accuracy = session.run(accuracy, feed_dict=feed_dict)
        print("迭代轮次: {0:>6}, 训练准确率: {1:>6.4%}".format(i, train_accuracy))

    session.run(optimizer, feed_dict=feed_dict)

迭代轮次:      0, 训练准确率: 5.0000%
迭代轮次:    500, 训练准确率: 94.0000%
迭代轮次:   1000, 训练准确率: 97.0000%
迭代轮次:   1500, 训练准确率: 98.0000%
迭代轮次:   2000, 训练准确率: 100.0000%
